In [2]:
from difflib import SequenceMatcher
import pandas as pd
import numpy as np

In [20]:
#READ DATA:
IMDB_DF = pd.read_csv("imdb.csv")
imdb_header = list(IMDB_DF.columns.values)

#imdb_header
tbl1 = np.array(imdb_header)

ROTTEN_TOMATOES_DF = pd.read_csv("rotten_tomatoes.csv")
rt_header = list(ROTTEN_TOMATOES_DF.columns.values)
#rt_header
tbl2 = np.array(rt_header)

['Id',
 'Name',
 'Year',
 'Release Date',
 'Director',
 'Creator',
 'Actors',
 'Cast',
 'Language',
 'Country',
 'Duration',
 'RatingValue',
 'RatingCount',
 'ReviewCount',
 'Genre',
 'Filming Locations',
 'Description']

In [60]:
Pref=np.zeros((len(tbl1),len(tbl2)))
pref_tbl1=np.zeros((len(tbl1),len(tbl2)),dtype=np.int8)
ipref_tbl1=np.zeros((len(tbl1),len(tbl2)),dtype=np.int8)
pref_tbl=np.zeros((len(tbl1),len(tbl2)),dtype=np.int8)
pref_tbl2=np.zeros((len(tbl2),len(tbl1)),dtype=np.int8)

In [74]:
#assign the matching ratio between tbl1 and tbl2
for i in range(len(tbl1)):
    for j in range(len(tbl2)):
        Pref[i,j] = np.int(100*SequenceMatcher(None,tbl1[i],tbl2[j]).ratio())

In [78]:
#assign the array indexs of Pref sorted by the actual values decending based on the lenght of tbl1 (13)
for i in range(len(tbl1)):
    #get indexs values of rows and store them in the rows as well
    ipref_tbl1[i,:] = np.argsort(Pref[i,:])
    ipref_tbl1[i,:] = ipref_tbl1[i,::-1]
    pref_tbl[i,:] = np.sort(Pref[i,:])
    pref_tbl[i,:] = pref_tbl[i,::-1]

In [89]:
#loop through the pref_tbl and check if the value of the matching ratio greater than certain threshold 
#then get the index of these cells, otherwise set value to -1
threshold = 30
for i in range(len(tbl1)):
    for j in range(len(tbl2)):
        if pref_tbl[i,j] >= threshold:
            pref_tbl1[i,j] = ipref_tbl1[i,j]
        else: pref_tbl1[i,j] = -1

In [91]:
new_pref_tbl1=[]

for i in range(len(tbl1)):
    new_pref_tbl1.append(list(filter(lambda c: c != -1 ,pref_tbl1[i,:] )))

In [117]:
content = {'name':'','index pref':0 , 'corespondance':[], 'preferences':[],'state':False,}
Rotten_tomatoes=[]

#assign the array indexs of Pref sorted by the actual values decending based on the lenght of tbl2 (15)
for i in range(len(tbl2)):
    #get indexs values of columns from Pref and store them in the rows
    pref_tbl2[i,:]=np.argsort(Pref[:,i])
    pref_tbl2[i,:]=pref_tbl2[i,::-1]
    Rotten_tomatoes.append({'name':tbl2[i],'index pref':0 , 'corespondance':pref_tbl2[i,:], 'preferences':[],'state':False,})
Rotten_tomatoes[3]

{'corespondance': array([ 3,  2,  6,  8,  5,  7,  1, 10,  4, 12,  9, 11,  0], dtype=int8),
 'index pref': 0,
 'name': 'Release Date',
 'preferences': [],
 'state': False}

In [115]:
Imdb = []

for i in range(len(tbl1)):
    Imdb.append({'name':tbl1[i],'index pref':0 , 'corespondance':[], 'preferences':new_pref_tbl1[i],'state':False,})
Imdb[3]

{'corespondance': [],
 'index pref': 0,
 'name': 'Name',
 'preferences': [1, 8],
 'state': False}

In [122]:
def marriage_algo():
    imdb_no = 0
    while imdb_no < len(tbl1):
        if len(Imdb[imdb_no]['preferences']) > 0:
            if Imdb[imdb_no]['state'] == False:
                pref_index = Imdb[imdb_no]['index pref']
                roto_no = Imdb[imdb_no]['preferences'][pref_index]
                imdb_no = match_pref(imdb_no,roto_no)
            else: imdb_no += 1
        else: imdb_no += 1
    return Imdb,Rotten_tomatoes

In [123]:
def match_pref(Ino,Rno):
    if Rotten_tomatoes[Rno]['state'] == False :
        Rotten_tomatoes[Rno]['state'] = True
        Imdb[Ino]['state'] = True
        Rotten_tomatoes[Rno]['index pref'] = find_index(Rotten_tomatoes,Rno , Ino)
        return Ino + 1
    else:
        sw = check_Rotten_pref(Ino,Rno)
        if Ino == 11:
            print(sw)
        if sw == -1 :
            if Imdb[Ino]['index pref'] < len(Imdb[Ino]['preferences']):#-----------------------------
                Imdb[Ino]['index pref'] += 1
                if Ino == 11:
                    print(Ino , '---------', Imdb[Ino]['index pref'])
                return Ino
            else:
                return Ino + 1
        else:
            Rotten_tomatoes[Rno]['state'] = True
            Imdb[Ino]['state'] = True
            Rotten_tomatoes[Rno]['index pref'] = find_index(Rotten_tomatoes,Rno , Ino)
            Imdb[sw]['state'] = False
            if Imdb[sw]['index pref'] < len(Imdb[Ino]['preferences']): #-------------------------
                Imdb[sw]['index pref'] += 1
                return sw
            else:
                return Ino+1

In [124]:
def find_index(table , row , value):
    for i in range(len(table[row]['preferences'])) :
        if table[row]['preferences'][i] == value:
            return i

In [125]:
def check_Rotten_pref(Ino,Rno):
    sw = find_index(Rotten_tomatoes,Rno,Ino)
    if Ino == 11:
        print(sw,Rotten_tomatoes[Rno]['index pref'])
    if sw < Rotten_tomatoes[Rno]['index pref']:
        return Rotten_tomatoes[Rno]['preferences'][Rotten_tomatoes[Rno]['index pref']]
    else:
        return -1

In [129]:
Imdb,Rotten_tomatoes = marriage_algo()

[{'corespondance': array([ 0, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1], dtype=int8),
  'index pref': None,
  'name': 'Id',
  'preferences': [],
  'state': True},
 {'corespondance': array([ 1,  8,  6, 10,  5,  7,  4,  2, 12,  3,  9, 11,  0], dtype=int8),
  'index pref': None,
  'name': 'Name',
  'preferences': [],
  'state': True},
 {'corespondance': array([ 2,  5, 10, 11, 12,  3,  6,  1,  9,  7,  4,  8,  0], dtype=int8),
  'index pref': None,
  'name': 'Year',
  'preferences': [],
  'state': True},
 {'corespondance': array([ 3,  2,  6,  8,  5,  7,  1, 10,  4, 12,  9, 11,  0], dtype=int8),
  'index pref': None,
  'name': 'Release Date',
  'preferences': [],
  'state': True},
 {'corespondance': array([ 4,  5,  7, 12, 10,  2,  3, 11,  6,  1,  8,  9,  0], dtype=int8),
  'index pref': None,
  'name': 'Director',
  'preferences': [],
  'state': True},
 {'corespondance': array([ 5,  4,  6,  7,  9,  2, 10,  3, 12,  8, 11,  1,  0], dtype=int8),
  'index pref': None,
  'name': 'Creator',


In [130]:
corresponacy = []
for i in range(len(Imdb)):
    if Imdb[i]['state'] == True:
        p = int(Imdb[i]['index pref'])
        j = Imdb[i]['preferences'][p]
        corresponacy.append(tuple([Imdb[i]['name'],Rotten_tomatoes[j]['name']]))

In [131]:
found = len(corresponacy)-1
print('corresponacies that our algorithm has found are:')
for i in range(len(corresponacy)):
    if i == 9:
        print(i,'. WRONG \t',corresponacy[i])
    else:
        print(i,'. CORRECT \t',corresponacy[i])
        
print('\nnumber of corresponacies that it found: ' , found)
print('number of true corresponacies that it found:' , found-1)
print('precision :',(found-1.0)/found)
print('recall:\t' ,(found-1)/10.0)

corresponacies that our algorithm has found are:
0 . CORRECT 	 ('Id', 'Id')
1 . CORRECT 	 ('Name', 'Name')
2 . CORRECT 	 ('YearRange', 'Year')
3 . CORRECT 	 ('ReleaseDate', 'Release Date')
4 . CORRECT 	 ('Director', 'Director')
5 . CORRECT 	 ('Creator', 'Creator')
6 . CORRECT 	 ('Cast', 'Cast')
7 . CORRECT 	 ('Duration', 'Duration')
8 . CORRECT 	 ('RatingValue', 'RatingValue')
9 . WRONG 	 ('ContentRating', 'RatingCount')
10 . CORRECT 	 ('Genre', 'Genre')
11 . CORRECT 	 ('Description', 'Description')

number of corresponacies that it found:  11
number of true corresponacies that it found: 10
precision : 0.9090909090909091
recall:	 1.0


In [85]:
import numpy,datetime,re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


C:\Users\ahmed\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [86]:
groups=['Short string','Long string','URL']

In [87]:
Imdb = IMDB_DF
Rotten_tomatoes = ROTTEN_TOMATOES_DF

Imdb = Imdb.sample(500).astype(str)
Rotten_tomatoes = Rotten_tomatoes.sample(500).astype(str)

imdb = Imdb.columns.values
rotten_tomatoes = Rotten_tomatoes.columns.values

In [88]:
def string_length(COL):
    short_s = 60
    summ = 0
    for i in COL:
        summ += len(str(i))
    ret = summ / len(COL)
    if ret < short_s:
        return True
    else :
        return False

def is_url(COL):
    no_url = 0.0
    search = "http://"
    match_rate = 0.0
    for i in COL:
        if re.search(search, str(i)):
            no_url += 1
    match_rate = no_url / len( COL) * 100
    if match_rate > 0.6:
        return True
    else:
        return False

In [89]:
def categorize_cols(cols_list,table):
    tablegroups={'Short string':[],'Long string':[],'URL':[]}
    for i in cols_list:
        tablegroups[find_group(table[i])].append(i)
    return tablegroups

def find_group(COL):
    if is_url(COL):
        return 'URL'
    elif string_length(COL):
        return 'Short string'
    else:
        return 'Long string'

In [90]:
Imdb_groups = categorize_cols(imdb,Imdb.dropna())
Rotten_tomatoes_groups = categorize_cols(rotten_tomatoes,Rotten_tomatoes.dropna())
IMgroup1 = Imdb[Imdb_groups['URL']].dropna()
IMgroup2 = Imdb[Imdb_groups['Short string']].dropna()
IMgroup3 = Imdb[Imdb_groups['Long string']].dropna()
RTgroup1 = Rotten_tomatoes[Rotten_tomatoes_groups['URL']].dropna()
RTgroup2 = Rotten_tomatoes[Rotten_tomatoes_groups['Short string']].dropna()
RTgroup3 = Rotten_tomatoes[Rotten_tomatoes_groups['Long string']].dropna()

In [94]:
def check_short_matching(COL1,COL2):
    rate = 0.0
    if str(COL1) == 'nan' or str(COL2) == 'nan' :
        return 0
    COL1 = str(COL1)
    COL2 = str(COL2)
    sw = (fuzz.SequenceMatcher(None,COL1, COL2).ratio())
    return sw #/(max(len(COL1),len(COL2))*1.0)

In [95]:
def check_match(st1,st2):
    str1 = np.array((st1.values).astype(str))
    str2 = np.array((st2.values).astype(str))
    ret = np.zeros((len(str1),len(str2)))
    for i in range(len(str1)):
        for j in range(len(str2)):
            ret[i,j] = check_short_matching(str1[i],str2[j])
    return numpy.average(ret)

In [96]:
def group1_sim(IMgroup1,RTgroup1):
    return []

In [98]:
def group2_sim(IMgroup2,RTgroup2):
    count = []
    dd = []
    ret = []
    v1 = []
    v2 = []
    for i in IMgroup2:
        dd = []
        for j in RTgroup2:
            g1 = IMgroup2[i]
            g2 = RTgroup2[j]
            v1 = (g1.values).astype(str)
            v2 = (g2.values).astype(str)
            dd.append([ 10000* (check_match(g1, g2)),i,j])
        ret += dd
        print(i,j)
    print('done')
    return ret

In [99]:
def group3_sim(IMgroup3,RTgroup3):
    count = []
    dd = []
    ret = []
    v1 = []
    v2 = []
    for i in IMgroup3:
        dd = []
        for j in RTgroup3:
            g1 = IMgroup3[i]
            g2 = RTgroup3[j]
            v1 = (g1.values).astype(str)
            v2 = (g2.values).astype(str)
            dd.append([ (10000* check_match(g1, g2)),i,j])
        ret += dd
        print(i,j)
    print('done')
    return ret

In [100]:
group1_redic=group1_sim(IMgroup1,RTgroup1)
group2_redic=group2_sim(IMgroup2,RTgroup2)
group3_redic=group3_sim(IMgroup3,RTgroup3)

Id Genre
Name Genre
YearRange Genre


KeyboardInterrupt: 

In [ ]:
group2_redic